In [1]:
import numpy as np
from pyfinite import ffield


F = ffield.FField(7, gen=0x83, useLUT=-1)

def Exponentiate(base,power):
    ans = base
    for i in range(1,power):
        ans = F.Multiply(ans,base)
    return ans


def LinearTransform(linmat,msg):
    ans = [0]*8
    for i in range(8):
        temp  = []
        mul = [F.Multiply(linmat[i][j],msg[i]) for j in range(8)]
        for k in range(8):
            temp.append(np.bitwise_xor(ans[k],mul[k]))
        ans = temp
    return ans

In [2]:
def decode_block(cipher):
  plain= ""
  for i in range(0,len(cipher),2):
      plain+=chr(16*(ord(cipher[i:i+2][0]) - ord('f')) + ord(cipher[i:i+2][1]) - ord('f'))
  return plain

In [3]:
#for diagonal elements
PossibleExponents = [[] for i in range(8)]   
possibleDiagonalVals=[[[] for i in range(8)] for j in range(8)]
input_file = open('plaintexts.txt','r')
output_file = open('ciphertexts.txt','r')
input = (input_file.readlines()[0]).strip().split(' ')
output = output_file.readlines()

input_string = []
for msg in input:
    input_string.append(decode_block(msg)[0])
#print(input_string)
#print(len(output))
output_string = []
for i in range(len(output)):
    x = []
    for msg in output[i].strip().split(' '):
        x.append(decode_block(msg)[i])
    output_string.append(x)
#print(output_string)
for k in range(8):
    for i in range(1, 127):
        for j in range(1, 128):
          flag = True
          for m in range(128):
            if(ord(output_string[k][m]) != Exponentiate(F.Multiply(Exponentiate(F.Multiply(Exponentiate(ord(input_string[m]), i), j), i), j), i)):
              flag = False
              break
          if(flag):
            PossibleExponents[k].append(i)
            possibleDiagonalVals[k][k].append(j)
print("Possible diagonal values: ")
print(possibleDiagonalVals)
print("Possible exponents: ")
print(PossibleExponents)


output_string = []
for i in range(len(output)-1):
    x = []
    for msg in output[i].strip().split(' '):
        x.append(decode_block(msg)[i+1])
    output_string.append(x)

for ind in range(7):
  for i in range(1, 128):
      for p1, e1 in zip(PossibleExponents[ind+1], possibleDiagonalVals[ind+1][ind+1]):
          for p2, e2 in zip(PossibleExponents[ind], possibleDiagonalVals[ind][ind]):
              for k in range(128):
                  flag = True
                  x1 = F.Multiply(Exponentiate(F.Multiply(Exponentiate(ord(input_string[k]), p2), e2), p2), i)
                  x2 = F.Multiply(Exponentiate(F.Multiply(Exponentiate(ord(input_string[k]), p2), i), p1), e1)
                  c1 = np.bitwise_xor(x1,x2)
                  if(ord(output_string[ind][k]) != Exponentiate(c1,p1)):
                      flag = False
                      break
              if flag:
                  PossibleExponents[ind+1] = [p1]
                  possibleDiagonalVals[ind+1][ind+1] = [e1]
                  PossibleExponents[ind] = [p2]
                  possibleDiagonalVals[ind][ind] = [e2]
                  possibleDiagonalVals[ind][ind+1] = [i]
print('===========================')
print(possibleDiagonalVals)
print(PossibleExponents)


Possible diagonal values: 
[[[84, 40, 49], [], [], [], [], [], [], []], [[], [122, 62, 70], [], [], [], [], [], []], [[], [], [119, 43, 5], [], [], [], [], []], [[], [], [], [68, 95, 12], [], [], [], []], [[], [], [], [], [47, 112, 96], [], [], []], [[], [], [], [], [], [38, 11, 58], [], []], [[], [], [], [], [], [], [27, 14], []], [[], [], [], [], [], [], [], [38, 92, 91]]]
Possible exponents: 
[[22, 37, 68], [26, 113, 115], [2, 38, 87], [17, 41, 69], [65, 92, 97], [29, 43, 55], [20, 108], [26, 113, 115]]
[[[84], [112], [], [], [], [], [], []], [[], [70], [30], [], [], [], [], []], [[], [], [43], [3], [], [], [], []], [[], [], [], [12], [104], [], [], []], [[], [], [], [], [112], [99], [], []], [[], [], [], [], [], [11], [88], []], [[], [], [], [], [], [], [27], [30]], [[], [], [], [], [], [], [], [38]]]
[[22], [115], [38], [69], [92], [43], [20], [26]]


In [4]:
def EAEAE (msg, lin_mat, exp_mat): # Defines EAEAE
  msg = [ord(m) for m in msg]
  res = [Exponentiate(msg[i], exp_mat[i]) for i in range(8)]
  res = LinearTransform(lin_mat, res)
  res = [Exponentiate(res[i], exp_mat[i]) for i in range(8)]
  res = LinearTransform(lin_mat, res)
  res = [Exponentiate(res[i], exp_mat[i]) for i in range(8)]
  return res


input_file = open('plaintexts.txt','r')
output_file = open('ciphertexts.txt','r')
input = input_file.readlines()
output = output_file.readlines()


input_string = []
for i in range(len(input)):
    x = []
    for msg in input[i].strip().split(' '):
        x.append(decode_block(msg))
    input_string.append(x)


output_string = []
for i in range(len(output)):
    x = []
    for msg in output[i].strip().split(' '):
        x.append(decode_block(msg))
    output_string.append(x)



for indexex in range(0,6):
    offset = indexex + 2
    
   
    exp_list = [e[0] for e in PossibleExponents]
    lin_trans_list = np.zeros((8,8),int)

    for i in range(8):
      for j in range(8):     
        if(len(possibleDiagonalVals[i][j]) != 0):
            lin_trans_list[i][j] = possibleDiagonalVals[i][j][0]
        else:
            lin_trans_list[i][j] = 0
          

    
    for index in range(8):
        if(index > (7-offset)):
            continue

        for i in range(127):
            lin_trans_list[index][index+offset] = i+1
            flag = True
            for inps, outs in zip(input_string[index], output_string[index]):
                x1 = EAEAE(inps, lin_trans_list, exp_list)[index+offset]
                x2 = outs[index+offset]
                if x1 != ord(x2):
                    flag = False
                    break
            if flag==True:
                possibleDiagonalVals[index][index+offset] = [i+1]
                
A = np.zeros((8,8),dtype='int')

for i in range(0,8):
    for j in range(0,8):
      if len(possibleDiagonalVals[j][i]) != 0:
       A[i][j] = possibleDiagonalVals[j][i][0]



E = exp_list

print(A)
print(E)

[[ 84   0   0   0   0   0   0   0]
 [112  70   0   0   0   0   0   0]
 [ 13  30  43   0   0   0   0   0]
 [102  16   3  12   0   0   0   0]
 [111  62   0 104 112   0   0   0]
 [ 25  50  25  51  99  11   0   0]
 [  8 123  23 103  25  88  27   0]
 [ 67   3  74  26  10  66  30  38]]
[22, 115, 38, 69, 92, 43, 20, 26]


In [5]:


Einverse = np.zeros((128, 128), dtype = int)

for base in range(0,128):
    temp = 1
    for power in range(1,127):
        result = F.Multiply(temp, base)
        Einverse[power][result] = base
        temp = result
       
inverseA = [[ 67,   0,   0,   0,   0,   0,   0,   0],
       [ 81,  77,   0,   0,   0,   0,   0,   0],
       [ 26,   5,  96,   0,   0,   0,   0,   0],
       [ 13,  15,  24,  94,   0,   0,   0,   0],
       [ 80,   4,  41, 120,  19,   0,   0,   0],
       [ 27,  99,  25,  92,  23,  74,   0,   0],
       [ 55,  83,  71,   5,  17, 112,  14,   0],
       [ 81,  30, 123,  87,  64, 127,  48,  56]]


In [6]:
password = "gsfojqmrimffismjfkjtkpkujlmjhjkp" #Encrypted password
block_size = 16
num_blocks = int(len(password) / block_size) # 2 blocks

def Einverse_calculator(block, E):
    return [Einverse[E[i]][block[i]] for i in range(8) ]

def Ainverse_calculator(block, A):
    transformed = []
    for row_num in range(0,8):
        elem_sum = 0
        for col_num in range(0,8):
            elem = F.Multiply(A[row_num][col_num], block[col_num])
            elem_sum = F.Add(elem, elem_sum)
        transformed+=[elem_sum]
    return transformed

decrypted_password = ""
for i in range(0,2): # Decipher both the blocks of the password
    elements = password[block_size*i:block_size*(i+1)]
    currentBlock = []
    for j in range(0,8):
        currentBlock+=[(ord(elements[2*j]) - ord('f'))*16 + (ord(elements[2*j+1]) - ord('f'))]
    
    EA = Ainverse_calculator(Einverse_calculator(currentBlock, E), inverseA)
    EAE = Einverse_calculator(EA, E)
    
    EAEA = Ainverse_calculator(EAE, inverseA)
    EAEAE = Einverse_calculator(EAEA, E)
    
    print("Inverse EAEAE values for block:",str(i)+": ",EAEAE)
    for ch in EAEAE:
        decrypted_password += chr(ch)
print("The password is",decrypted_password)
    

Inverse EAEAE values for block: 0:  [118, 116, 111, 107, 100, 112, 109, 119]
Inverse EAEAE values for block: 1:  [101, 111, 48, 48, 48, 48, 48, 48]
The password is vtokdpmweo000000
